This file determines which bounds (lower vs upper) proportionally. 

In [1]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

# Move to the parent directory
parent_directory = os.path.dirname(current_directory)
os.chdir(parent_directory)

# Print the updated working directory
updated_directory = os.getcwd()
print("Updated Directory:", updated_directory)

Current Directory: /mnt/c/Users/rjmay/code/research_ms/alt_distr
Updated Directory: /mnt/c/Users/rjmay/code/research_ms


In [2]:
! source ../venv/bin/activate

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import pickle
import time

from modules.constants import sample_sizes
sample_sizes  = np.array(sample_sizes)*2

In [3]:
sample_sizes

array([  100,   160,   250,   400,   630,  1000,  1600,  2500,  4000,
        6300, 10000])

In [51]:

dim_str = "15" # TODO change me 


In [52]:
# BER_values = {'beta_beta': 0.10937546875016925, 'paired_uniforms': .1, 'normal_beta': 0.0726617598018901,
#                'uniform_normal': 0.15350691875939732}

BER_values = {'gm': 0.13868461929442688, 'paired_uniforms': .1, 'normal_beta': 0.0726617598018901,
               'uniform_normal': 0.15350691875939732}

In [53]:
for k in BER_values:
    print((k))

gm
paired_uniforms
normal_beta
uniform_normal


In [54]:
# wanted_columns = ['Dp', 'Bha',
#        'Bha_knn', 'inf', 'enDive']

wanted_columns = ['Dp_lower', 'Dp_upper', 'Bha_lower', 'Bha_upper', 'Bha_knn_lower',
       'Bha_knn_upper', 'inf_lower', 'inf_upper',
       'enDive_lower', 'enDive_upper',]

### Simulation stage

In [55]:
# from modules.multi_bounds_v3 import bounds_class
# # from modules.Bhatt_knn_func import knn_num_calc
# from modules.data_gen import data_gen

In [56]:
all_obj = {}

for k in BER_values:
    file_path = 'sim_data/' + k + dim_str + '.pkl'
    print(file_path, end = '\t')
    #Open the file in binary read mode
    with open(file_path, 'rb') as file:
        # Use pickle.load to deserialize and load the list of objects from the file
        loaded_objects = pickle.load(file)

    print('Loaded object successfully')
    
    all_obj[k] = loaded_objects



sim_data/gm15.pkl	Loaded object successfully
sim_data/paired_uniforms15.pkl	Loaded object successfully
sim_data/normal_beta15.pkl	Loaded object successfully
sim_data/uniform_normal15.pkl	Loaded object successfully


In [57]:
all_obj['gm']

[[<modules.multi_bounds_parfor.bounds_class at 0x7f4d9e0550c0>,
 [50, 80, 125, 200, 315, 500, 800, 1250, 2000, 3150, 5000]]

### Using the data



In [58]:
# 
dpi = 250


In [59]:
validity_dict = {}

for k in all_obj:
    
    
    temp_bound_obj_lst  = all_obj[k][0]

    lst = [obj.validity(BER_values[k]) for obj in temp_bound_obj_lst ]
    df_valid = pd.DataFrame(lst, index = sample_sizes)

    good_columns  = df_valid[wanted_columns]

    short_df = pd.concat([good_columns.head(1).tail(1), good_columns.tail(1)])
    
    validity_dict[k] = short_df



In [60]:
# final_df = pd.concat(
#     [df.assign(Key=key) for key, df in validity_dict.items()],
#     ignore_index=True
# ) 

# final_df

In [61]:
#  Assuming validity_dict is populated as per your previous code
final_df_list = []

for key, df in validity_dict.items():
    # Create a new column combining the key and index
    df = df.copy()  # Avoid modifying the original DataFrame
    df['Key'] = [f'{key}_{idx}' for idx in df.index]
    
    # Append the DataFrame with the new Key column to the list
    final_df_list.append(df)

# Concatenate the list of DataFrames into one final DataFrame
final_validity = pd.concat(final_df_list, ignore_index=True)

# Display the final DataFrame
final_validity


final_validity.columns

Index(['Dp_lower', 'Dp_upper', 'Bha_lower', 'Bha_upper', 'Bha_knn_lower',
       'Bha_knn_upper', 'inf_lower', 'inf_upper', 'enDive_lower',
       'enDive_upper', 'Key'],
      dtype='object')

In [62]:


order_columns = [
     'Bha_lower', 'Bha_upper', 'Bha_knn_lower',
       'Bha_knn_upper', 'inf_lower', 'inf_upper', 'Dp_lower', 'Dp_upper', 'enDive_lower',
       'enDive_upper', 'Key'
]

In [63]:
combined_sorted = final_validity[order_columns]

combined_sorted.index = combined_sorted.Key

combined_sorted= combined_sorted.drop(columns=['Key'])


combined_sorted.round(3)

,Bha_lower,Bha_upper,Bha_knn_lower,Bha_knn_upper,inf_lower,inf_upper,Dp_lower,Dp_upper,enDive_lower,enDive_upper
Key,,,,,,,,,,
gm_100,0.000,1.0,0.968,0.962,1.0,0.0,0.046,1.000,0.488,0.520
gm_10000,0.000,1.0,1.000,1.000,1.0,0.0,0.066,1.000,0.586,0.508
paired_uniforms_100,0.788,1.0,0.802,0.998,1.0,0.0,0.000,1.000,0.496,0.524
paired_uniforms_10000,1.000,1.0,0.798,1.000,1.0,0.0,0.000,1.000,0.612,0.652
normal_beta_100,0.230,1.0,0.238,1.000,1.0,0.0,0.000,1.000,0.476,0.550
normal_beta_10000,1.000,1.0,0.000,1.000,1.0,0.0,0.000,1.000,0.152,0.954
uniform_normal_100,0.224,1.0,1.000,0.836,1.0,0.0,0.370,0.998,0.512,0.518
uniform_normal_10000,1.000,1.0,1.000,1.000,1.0,0.0,1.000,1.000,0.794,0.532
